## Happiness Score from the [World Happiness Report](https://worldhappiness.report/)
In this notebook we will use the "World Happiness Report" datasets from 2015 to 2023. This dataset is based on the yearly Gallup World Poll which surveys around 100,000 people from 130 countries. The main metric is the Cantril ladder which asks respondents to think of a ladder and their standing on it with the best possible life for them being a 10 and the worst possible life being a 0. We select the five countries with the highest variance across the years with respect to the Cantrill Ladder Score.

The report looks into six key factors that could influence the Cantril ladder evaluation. These extrapolate the evaluation to national metrics: GDP per capita, Healthy Life Expectancy, Generosity, Social Support, Freedom to make life choices, Perceptions of corruption, and Dystopia. We select the eight countries with highest variance across Healthy Life Expectancy, Generosity, and Social Support from 2020 to 2023.

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [20]:
directory_path = './WHR'
# Get the list of all files in the directory
file_list = os.listdir(directory_path)
file_list.sort()
file_list

['2015_Chapter2OnlineData_Expanded-with-Trust-and-Governance.xlsx',
 '2016_Online-data-for-chapter-2-whr-2016.xlsx',
 '2017_online-data-chapter-2-whr-2017.xlsx',
 '2018_WHR2018Chapter2OnlineData.xls',
 '2019_Chapter2OnlineData.xls',
 '2020_WHR20_DataForFigure2.1.xls',
 '2021_DataForFigure2.1WHR2021C2.xls',
 '2022_Appendix_2_Data_for_Figure_2.1.xls',
 '2023_DataForFigure2.1WHR2023.xls',
 'Explainedby']

In [18]:
DF.columns

Index(['Country name', 'Ladder score', 'Standard error of ladder score',
       'upperwhisker', 'lowerwhisker', 'Logged GDP per capita',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Ladder score in Dystopia',
       'Explained by: Log GDP per capita', 'Explained by: Social support',
       'Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption',
       'Dystopia + residual'],
      dtype='object')